In [1]:
# conda install altair --channel conda-forge

In [2]:
import os
import re
import csv
import sys
import nltk
import pprint
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

input_file = 'USvideos.csv'
pos_file = 'parts_of_speech.csv'

with open(input_file, 'r') as youtube_data:
    masterdata_csv = list(csv.reader(youtube_data))

with open(pos_file, 'r') as parts_of_speech:
    pos_load = csv.reader(parts_of_speech)
    pos_dict = {}
    for row in pos_load:
        pos_dict[row[0]] = row[1]

def run_query(query):
    return pd.read_sql_query(query,db)

def open_db(database):
    db = sqlite3.connect(database)
    db.execute('PRAGMA foreign_keys = ON;')
    return db

In [3]:
# youtube_pd = pd.read_csv('USvideos.csv', index_col = None, na_values = ['NA'])
# masterdata_pd.head(n = 5)
# masterdata_pd.tail(n =5)
# filtered_yt = youtube_pd[(youtube_pd.title == 'something') & (youtube_pd.views > something else)]
#filtered_yt = youtube_pd.filter(items = ['title', 'views'])
# youtube_pd.describe() will summarize your data for you
# youtube_pd['likes'].describe()
# pandas has a merge function, like SQL joins. pd.merge(), supports left right inner outer
# .pivot allows for the creation or pivot tables by indicating a row x colum array (child poverty shown with city as the rows and year as the columns)

# Can I write derrived data back into a panda data set?
# Where does this fit in with the requirements to use SQL?
# How does this work library to library? Can I feed panda data into matplotlib? By subsetting, or indexing

Pandas - Panel Data

Rows are a ranges, columns are variables
pd.read_csv()
put the csv in the repo -- they want to see you download the data to the current working directory


#### Create database and master table

In [4]:
try : 
    os.remove('youtube.db')
except FileNotFoundError:
    pass

db = open_db('youtube.db')
c = db.cursor()

create_statement_master = '''
    CREATE TABLE tblMasterData (
    id INTEGER AUTOIMCREMENT PRIMARY KEY,
    video_id TEXT,
    trending_date TEXT,
    title TEXT,
    channel_title TEXT,
    category_id INTEGER,
    publish_time INTEGER,
    tags TEXT,
    views INTEGER,
    likes INTEGER,
    dislikes INTEGER,
    comment_count INTEGER,
    thumbnail_link TEXT,
    comments_disabled TEXT,
    ratings_disabled TEXT,
    video_error_or_removed TEXT,
    description TEXT)
    '''

c.execute(create_statement_master)

db.commit() # put in a finally (try, except, finally)

db.close() # put in a finally (try, except, finally)

#### Save column headers into a dictionary and delete column header row

In [5]:
headers = masterdata_csv[0]
headers_dict = {}

for count, value in enumerate(headers, 1):
    headers_dict[value] = count -1
# to keep from 0 indexing my columns

del masterdata_csv[0]

#### Load data into master table

In [6]:
db = open_db('youtube.db')
c = db.cursor()

c.executemany('''
INSERT INTO tblMasterData
(video_id,
trending_date,
title,
channel_title,
category_id,
publish_time,
tags,
views,
likes,
dislikes,
comment_count,
thumbnail_link,
comments_disabled,
ratings_disabled,
video_error_or_removed,
description)
VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
''', 
masterdata_csv)

db.commit()

master_limit5 = run_query('SELECT * FROM tblMasterData LIMIT 5;')

db.close()

master_limit5

,id,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,None,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,None,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,None,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,None,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,None,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


#### After reviewing master data, create two tables based on function -- one static, one transactional

In [7]:
db = open_db('youtube.db')
c = db.cursor()

create_statement_video = '''
    CREATE TABLE tblVideos (
    video_id TEXT PRIMARY KEY NOT NULL,
    title TEXT NOT NULL,
    channel_title TEXT NOT NULL,
    publish_time INTEGER NOT NULL,
    tags TEXT NOT NULL,
    thumbnail_link TEXT NOT NULL,
    comments_disabled TEXT NOT NULL,
    ratings_disabled TEXT NOT NULL,
    video_error_or_removed TEXT NOT NULL,
    description TEXT NOT NULL)
    '''

c.execute(create_statement_video)

create_statement_time = '''
    CREATE TABLE tblTime (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    video_id TEXT NOT NULL,
    trending_date TEXT NOT NULL,
    views INTEGER NOT NULL,
    likes INTEGER NOT NULL,
    dislikes INTEGER NOT NULL,
    comment_count INTEGER NOT NULL,
        FOREIGN KEY(video_id) REFERENCES tblVideos(video_id))
    '''

c.execute(create_statement_time)

db.commit()

db.close()

#### Date is stored in hard-to-read format. Transform date column.

In [8]:
for row in masterdata_csv:
    date = row[headers_dict['trending_date']]
    updated_date = "20" + date[:2]
    updated_date += "-"
    updated_date += date[6:]
    updated_date += "-"
    updated_date += date[3:5]
    row[headers_dict['trending_date']] = updated_date

#### Titles contain characters + and &. These will not be parseable. Replacing with 'and'.

In [9]:
for row in masterdata_csv:
    title = row[headers_dict['title']]
    updated_title = re.sub('&|\+', 'and', title)
    row[headers_dict['title']] = updated_title

#### Separate CSV file into lists to be loaded to each table

In [10]:
video_data = []
time_data = []

for row in masterdata_csv:
    video_entry = [row[headers_dict['video_id']], 
                   row[headers_dict['title']], 
                   row[headers_dict['channel_title']], 
                   row[headers_dict['publish_time']],
                   row[headers_dict['tags']],
                   row[headers_dict['thumbnail_link']],
                   row[headers_dict['comments_disabled']],
                   row[headers_dict['ratings_disabled']],
                   row[headers_dict['video_error_or_removed']],
                   row[headers_dict['description']]
                  ]
    
    video_data.append(video_entry)
    
    time_entry = [row[headers_dict['video_id']], 
                  row[headers_dict['trending_date']], 
                  row[headers_dict['views']], 
                  row[headers_dict['likes']],
                  row[headers_dict['dislikes']],
                  row[headers_dict['comment_count']]
                 ]
    
    time_data.append(time_entry)

#### Load data to tables

In [11]:
db = open_db('youtube.db')
c = db.cursor()

c.executemany('''
        INSERT OR REPLACE INTO tblVideos
        (video_id,
        title,
        channel_title,
        publish_time,
        tags,
        thumbnail_link,
        comments_disabled,
        ratings_disabled,
        video_error_or_removed,
        description)
        VALUES (?,?,?,?,?,?,?,?,?,?)
        ''', 
        video_data)

db.commit()
    
c.executemany('''
        INSERT INTO tblTime
        (video_id,
        trending_date,
        views,
        likes,
        dislikes,
        comment_count)
        VALUES (?,?,?,?,?,?)
        ''', 
        time_data)

db.commit()

videos_limit5 = run_query('SELECT * FROM tblVideos LIMIT 5;')

time_limit5 = run_query ('SELECT * FROM tblTime LIMIT 5;')

db.close()

# to view a sample, uncomment below
# videos_limit5
# time_limit5

#### Test for foreign key failure  

In [12]:
db = open_db('youtube.db')
c = db.cursor()

try:
    c.execute("INSERT INTO tblTime VALUES (?, ?, ?, ?, ?, ?, ?)", (None, "testy", "18.11.11", 1, 1, 1, 1))
except:
    print(sys.exc_info())

db.commit()
db.close()

(<class 'sqlite3.IntegrityError'>, IntegrityError('FOREIGN KEY constraint failed',), <traceback object at 0x1a14f01e48>)


###### ^ look at that! It failed!!! :)

#### Titles appear to come in multiple segments, divided by special characters. Create Segments table to store segments of each title for analysis

In [13]:
db = open_db('youtube.db')
c = db.cursor()

create_statement_segments = '''
    CREATE TABLE tblSegments (
    segment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    video_id TEXT NOT NULL,
    segment_text TEXT NOT NULL,
    segment_structure TEXT,
        FOREIGN KEY(video_id) REFERENCES tblVideos(video_id))
    '''

c.execute(create_statement_segments)

db.commit()
db.close()


#### Define classes to facilitate analaysis...and for practice

In [14]:
class Video:
    
    def __init__(self, video_id, title):
        self.video_id = video_id
        self.title = title
    
    def longest_word(self):
        longest_length = 0
        longest_word = ''
        for word in self.title:
            if len(word) > longest_length:
                longest_length = len(word)
                longest_word = word
            else:
                continue
        return longest_word

    def title_segments(self):
        list = re.compile("(?:\||(?:\s-\s)|—|:|\(|\)|\[|\]|{|})+").split(self.title)
        # ?: indicates a non-capture group so delimiters aren't saved. Now I know.
        list = filter(lambda x: x != None, list)
        #  python is returning None where the delimiter was. Removing Nones.
        segments = []
        for text in list:
            text = text.strip()
            if text == "":
                continue
            segments.append(Segment(self.video_id, text))
        return segments

    
class Segment:
    
    def __init__(self, video_id, text):
        self.video_id = video_id
        self._text = text
        
    def text(self):
        return re.sub('[^A-Za-z0-9\s\-\']+', '', self._text)
    
    def words(self):
        list = self.text().split()
        list = filter(lambda x: x != None, list)
        #  python is returning None where the delimiter was. Removing Nones.
        segments = []
        for item in list:
            text = item.strip()
            if text == "":
                continue
            segments.append(text)
        return segments
    
    def parts_of_speech(self):
        text = nltk.word_tokenize(self.text())
        tagged_text = nltk.pos_tag(text)
        #  creates list of tuples with (word, part of speech)
        return [item[1] for item in tagged_text]

class Title_glob:
    
    def __init__(self, glob):
        self.glob = glob
    
    def word_list(self):
        return self.glob
    
    def frequency_distribution(self):
        return nltk.FreqDist(self.glob)
    
    def lexical_diversity(self):
        return len(set(self.glob)) / len(self.glob)

#### Select data from Videos table to parse into segments and commit to Segments table

In [15]:
db = open_db('youtube.db')
c = db.cursor()

c.execute("SELECT video_id, title FROM tblVideos;")
titles_list = c.fetchall()

db.commit()
db.close()

In [16]:
segment_entries=[]

for row in titles_list:
    video = Video(row[0], row[1])
    segments = video.title_segments()
    for segment in segments:
        segment_entry = []
        segment_entry.append(segment.video_id)
        segment_entry.append(segment.text())
        segment_entry.append(", ".join(segment.parts_of_speech()))
        segment_entries.append(segment_entry)
        
# it thinks they're all proper nouns. title probs.

In [17]:
db = open_db('youtube.db')
c = db.cursor()

c.executemany('''
        INSERT INTO tblSegments
        (video_id,
        segment_text,
        segment_structure)
        VALUES (?,?, ?)
        ''', 
        segment_entries)

select_segments = run_query('SELECT * FROM tblSegments WHERE segment_id>=(abs(random()) % (SELECT max(segment_id)FROM tblSegments)) LIMIT 5')

db.commit()
db.close()

select_segments

,segment_id,video_id,segment_text,segment_structure
0,10166,RxvcH25WThg,ESPN,NN
1,10167,hYOa56nwx5Y,What Are Fever Dreams,"WP, VBP, NNP, NNP"
2,10168,4_eVvzrTV-I,Shaquem Griffin Gets Selected by the Seattle S...,"NNP, NNP, NNP, VBN, IN, DT, NNP, NNP"
3,10169,4_eVvzrTV-I,2018 NFL Draft,"CD, NNP, NNP"
4,10170,lE0eWRdSrY4,Koreas,NNS


In [18]:
db = open_db('youtube.db')
c = db.cursor()

c.execute("SELECT video_id, segment_text, segment_id FROM tblSegments LIMIT 5;")
segments_list = c.fetchall()

db.commit()
db.close()

Concatenate all segments for analysis as a whole

In [19]:
def lower_case(list):
    return [w.lower() for w in list]

all_segment_words = []

for item in segments_list:
    segment = Segment(item[0], item[1])
    segment_words = segment.words()
    for word in segment_words: 
        all_segment_words.append(word)

all_segment_words = lower_case(all_segment_words)

all_words = Title_glob(sorted(all_segment_words))
print(all_words.word_list())

all_tokens = Title_glob(sorted(set(all_segment_words)))
print(all_tokens.word_list())

['10', '103-yd', 'before', "can't-miss", 'dceu', 'denver', 'dion', 'every', 'highlights', 'justice', 'kick', 'league', "lewis'", 'movie', 'nfl', 'play', 'return', 'td', 'tldw', 'vs', 'wk']
['10', '103-yd', 'before', "can't-miss", 'dceu', 'denver', 'dion', 'every', 'highlights', 'justice', 'kick', 'league', "lewis'", 'movie', 'nfl', 'play', 'return', 'td', 'tldw', 'vs', 'wk']


The X most common words

In [20]:
# freq_dist = title_glob.frequency_distribution()
# print(freq_dist.most_common(30))

The length of the set of unique words / the length of the text. Fairly Unique!

In [21]:
# title_glob.lexical_diversity()

To do:
- concat titles for fun analysis
- search nltk for stuff to make classes work

Fun stuff to report:
- Avg. title segments per title
- Longest word per title
